You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# pip install crewai crewai-tools
# %pip install 'crewai[tools]'

In [11]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [12]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [13]:
import os
from utils import pretty_print_result
# from utils import get_openai_api_key, pretty_print_result
# from utils import get_serper_api_key
import dotenv

dotenv.load_dotenv()
# openai_api_key = get_openai_api_key()
# os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY")

google_api_key = os.getenv("GOOGLE_API_KEY")


In [14]:
from crewai import LLM

llm = LLM(
    model="gemini/gemini-2.0-flash-001",
    temperature=0.7,
    # vertex_credentials=vertex_credentials_json
)

In [15]:
# import os
# from utils import get_openai_api_key

# openai_api_key = get_openai_api_key()
# os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## 에이전트 생성 (Creating Agents)

- 에이전트를 정의하고, `role` (역할), `goal` (목표) 및 `backstory` (배경 이야기)를 제공하세요.
- LLM (Large Language Model, 대규모 언어 모델)은 역할극을 할 때 성능이 더 좋은 것으로 알려져 있습니다.

### 에이전트: 기획자 (Planner)

**참고**: _여러 문자열 (multiple strings)_ 을 사용하는 이점:
```Python
varname = "텍스트의 첫 번째 줄 (line 1 of text)"
          "텍스트의 두 번째 줄 (line 2 of text)"
```

반면 _삼중 따옴표 독스트링 (triple quote docstring)_ 은:
```Python
varname = """텍스트의 첫 번째 줄 (line 1 of text)
             텍스트의 두 번째 줄 (line 2 of text)
          """
```
이러한 공백 (whitespaces) 과 줄바꿈 문자 (newline characters) 를 추가하는 것을 피할 수 있으며, 이는 LLM (Large Language Model) 에 전달하기에 더 나은 형식입니다.

In [16]:
planner = Agent(
    role="콘텐츠 기획자 (Content Planner)",
    goal="{topic}에 대한 흥미롭고 사실에 기반한 콘텐츠를 기획합니다 (Plan engaging and factually accurate content on {topic})",
    backstory="당신은 {topic}에 대한 블로그 기사를 기획하고 있습니다 (You're working on planning a blog article about the topic: {topic}). "
              "당신은 독자들이 무언가를 배우고 정보에 입각한 결정을 내리는 데 도움이 되는 정보를 수집합니다 (You collect information that helps the audience learn something and make informed decisions). "
              "당신의 작업은 콘텐츠 작성자 (Content Writer)가 이 주제에 대한 기사를 작성하는 기초가 됩니다 (Your work is the basis for the Content Writer to write an article on this topic).",
    allow_delegation=False,
    llm=llm,
    verbose=True
)

### Agent: Writer

In [17]:
writer = Agent(
    role="콘텐츠 작성자 (Content Writer)",
    goal="주제: {topic}에 대한 통찰력 있고 사실에 근거한 의견 (opinion)을 작성합니다.",
    backstory="당신은 주제: {topic}에 대한 새로운 의견 (opinion)을 작성하고 있습니다. "
              "당신은 콘텐츠 기획자 (Content Planner)의 작업을 기반으로 글을 쓰며, 콘텐츠 기획자는 개요 (outline)와 주제에 대한 관련 맥락 (context)을 제공합니다. "
              "당신은 콘텐츠 기획자가 제공하는 개요 (outline)의 주요 목표와 방향을 따릅니다. "
              "또한 객관적이고 공정한 통찰력을 제공하고 콘텐츠 기획자가 제공하는 정보를 통해 이를 뒷받침합니다. "
              "당신은 당신의 진술이 객관적인 진술이 아닌 의견 (opinion)일 경우, 당신의 의견 (opinion)에서 이를 인정합니다.",
    allow_delegation=False,
    llm=llm,
    verbose=True
)

### Agent: Editor

In [18]:
editor = Agent(
    role="편집자 (Editor)",
    goal="제공된 블로그 게시물을 편집하여 조직의 글쓰기 스타일에 맞춥니다 (Edit a given blog post to align with the writing style of the organization).",
    backstory="당신은 콘텐츠 작성자 (Content Writer)로부터 블로그 게시물을 받는 편집자입니다 (You are an editor who receives a blog post from the Content Writer). "
              "당신의 목표는 블로그 게시물을 검토하여 저널리즘의 모범 사례 (journalistic best practices)를 따르는지 확인하고, "
              "의견이나 주장을 제공할 때 균형 잡힌 관점 (balanced viewpoints)을 제공하며, "
              "가능한 경우 주요 논란이 되는 주제 (major controversial topics)나 의견 (opinions)을 피하는 것입니다.",
    allow_delegation=False,
    llm=llm,
    verbose=True
)


## 작업 생성 (Creating Tasks)

- 작업을 정의하고 `description` (설명), `expected_output` (예상 결과물) 및 `agent` (에이전트)를 제공하십시오.

### Task: Plan

In [19]:
plan = Task(
    description=(
        "1. {topic}에 대한 최신 트렌드, 주요 참여자 (key players) 및 주목할 만한 뉴스 (noteworthy news)를 우선적으로 고려합니다.\n"
        "2. 대상 독자 (target audience)를 식별하고, 그들의 관심사 (interests)와 고충 (pain points)을 고려합니다.\n"
        "3. 소개 (introduction), 주요 요점 (key points) 및 행동 촉구 (call to action)를 포함한 자세한 콘텐츠 개요 (content outline)를 개발합니다.\n"
        "4. SEO 키워드 (SEO keywords)와 관련된 데이터 (data) 또는 출처 (sources)를 포함합니다."
    ),
    expected_output="개요 (outline), 청중 분석 (audience analysis), SEO 키워드 (SEO keywords) 및 리소스 (resources)가 포함된 포괄적인 콘텐츠 계획 문서 (content plan document)입니다.",
    agent=planner,
)

### Task: Write

In [20]:
write = Task(
    description=(
        "1. 콘텐츠 계획 (content plan)을 활용하여 {topic}에 대한 설득력 있는 블로그 게시물 (blog post)을 작성합니다.\n"
        "2. SEO 키워드 (SEO keywords)를 자연스럽게 통합합니다.\n"
        "3. 섹션/부제 (Sections/Subtitles)의 이름이 매력적인 방식으로 적절하게 지정되었는지 확인합니다.\n"
        "4. 게시물이 매력적인 소개 (introduction), 통찰력 있는 본문 (insightful body) 및 요약 결론 (summarizing conclusion)으로 구성되었는지 확인합니다.\n"
        "5. 문법적 오류 (grammatical errors)가 없는지, 브랜드의 목소리 (brand's voice)와 일치하는지 교정합니다 (Proofread)."
    ),
    expected_output="잘 작성된 블로그 게시물 (blog post)을 마크다운 형식 (markdown format)으로 작성하여 게시할 준비를 합니다. 각 섹션은 2~3개의 단락 (paragraphs)으로 구성되어야 합니다.",
    agent=writer,
)

### Task: Edit

In [21]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [32]:
# crew = Crew(
#     agents=[planner, writer, editor],
#     tasks=[plan, write, edit],
#     verbose=2
# )

crew = Crew(
    # agents=[research_agent, writer_agent],
    # tasks=[research_task, write_article_task],
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],    
    verbose=True
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [33]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

# Agent: Content Planner
## Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


# Agent: Content Planner
## Final Answer: 
```
## Content Plan: Artificial Intelligence - Understanding the Present and Shaping the Future

**1. Target Audience Analysis:**

*   **Primary Audience:**
    *   Professionals in various industries (e.g., marketing, finance, healthcare, manufacturing) seeking to understand how AI can impact their work and businesses.
    *   Entrepreneurs and business leaders looking for opportunities to leverage AI for innovation and growth.
    *   Students and recent graduates interested in pursuing careers in AI or related fields.
*   **Secondary Audience:**
    *   Tech enthusiasts and early adopte

- Display the results of your execution as markdown in the notebook.

In [43]:
result

CrewOutput(raw="```markdown\n# Artificial Intelligence: Understanding the Present and Shaping the Future\n\nImagine a world where machines can learn, adapt, and solve problems like humans. That world is no longer a distant dream; it's rapidly becoming our reality, thanks to Artificial Intelligence (AI). AI is projected to contribute a staggering $15.7 trillion to the global economy by 2030, highlighting its significance as a transformative technology. But what exactly is AI, and why should you care?\n\nAt its core, Artificial Intelligence refers to the simulation of human intelligence in machines programmed to think and act like humans. This encompasses a broad range of techniques, including Machine Learning (ML), where systems learn from data without explicit programming; Deep Learning, a subset of ML that uses artificial neural networks with multiple layers to analyze data; and Natural Language Processing (NLP), which enables machines to understand and process human language. Underst

In [44]:
from IPython.display import Markdown
Markdown(result.raw)

```markdown
# Artificial Intelligence: Understanding the Present and Shaping the Future

Imagine a world where machines can learn, adapt, and solve problems like humans. That world is no longer a distant dream; it's rapidly becoming our reality, thanks to Artificial Intelligence (AI). AI is projected to contribute a staggering $15.7 trillion to the global economy by 2030, highlighting its significance as a transformative technology. But what exactly is AI, and why should you care?

At its core, Artificial Intelligence refers to the simulation of human intelligence in machines programmed to think and act like humans. This encompasses a broad range of techniques, including Machine Learning (ML), where systems learn from data without explicit programming; Deep Learning, a subset of ML that uses artificial neural networks with multiple layers to analyze data; and Natural Language Processing (NLP), which enables machines to understand and process human language. Understanding these core concepts is crucial to navigating the complex world of AI. This article will explore the latest trends in AI, its key applications across industries, and the ethical considerations surrounding its development and deployment.

## The Cutting Edge: Latest Trends in Artificial Intelligence

The field of AI is constantly evolving, with new breakthroughs emerging at an astonishing pace. Staying abreast of these trends is crucial for anyone looking to understand the present and future of technology. Let's delve into some of the most exciting developments.

**Generative AI: The Creative Revolution**

Generative AI is taking the world by storm, with its ability to create new content, from text and images to code and music. Models like DALL-E 2, Midjourney, Stable Diffusion, ChatGPT, and Bard are democratizing creativity, enabling anyone to generate high-quality content with simple text prompts. This has profound implications for content creation, marketing, design, and even software development, where AI can automate the generation of code snippets.

**Large Language Models (LLMs): Mastering Human Language**

Advancements in Large Language Models (LLMs) are revolutionizing Natural Language Processing (NLP). Models like GPT-4, LaMDA, and LLaMA can understand and generate human language with remarkable fluency and coherence. This is powering a new generation of chatbots, virtual assistants, language translation tools, and text summarization applications, making human-computer interaction more seamless and intuitive.

**AI-powered Automation: Efficiency Unleashed**

AI is increasingly being used to automate tasks across various industries, boosting efficiency and productivity. Robotic Process Automation (RPA) and intelligent automation platforms are streamlining workflows in manufacturing, logistics, customer service, and finance. By automating repetitive and mundane tasks, AI allows humans to focus on more creative and strategic work.

**Edge AI: Intelligence at the Source**

Edge AI brings AI processing closer to the data source, enabling real-time decision-making and reducing reliance on cloud connectivity. This is particularly beneficial in applications like AI-powered cameras, autonomous vehicles, and smart sensors, where low latency and increased privacy are critical. Edge AI is driving innovation in IoT, industrial automation, and security.

**Explainable AI (XAI): Unveiling the Black Box**

As AI systems become more complex, the need for transparency and explainability is growing. Explainable AI (XAI) focuses on developing techniques to make AI models more interpretable, allowing users to understand how AI systems arrive at their decisions. This is particularly important in high-stakes applications like healthcare, finance, and legal, where trust and accountability are paramount.

## The AI Powerhouses: Key Players Shaping the Landscape

The Artificial Intelligence landscape is populated by a diverse range of players, from tech giants to specialized AI companies and research institutions. Understanding who these key players are and what they are working on is crucial to understanding the direction of AI development.

**Big Tech Companies: The AI Titans**

Companies like Google (DeepMind), Microsoft, Amazon, and Meta (Facebook) are investing heavily in AI research and development, integrating AI into their products and services. Google is focused on fundamental AI research through DeepMind, while Microsoft is building a comprehensive AI platform and integrating AI into its cloud computing services. Amazon is leveraging AI for e-commerce and robotics, and Meta is exploring AI for social media and virtual reality.

**AI-Specific Companies: The Innovators**

Companies like OpenAI, Nvidia, DataRobot, and UiPath are pushing the boundaries of AI innovation. OpenAI is known for its generative AI models like the GPT series and DALL-E, while Nvidia is a leading provider of AI hardware (GPUs) and software platforms. DataRobot offers an automated machine learning platform, and UiPath specializes in Robotic Process Automation (RPA) software.

**Research Institutions: The Academic Pioneers**

Leading research institutions like MIT, Stanford, and Carnegie Mellon University are at the forefront of AI research, driving breakthroughs in areas like machine learning, computer vision, and natural language processing. These institutions are training the next generation of AI researchers and engineers.

## AI in the News: Noteworthy Developments

The world of Artificial Intelligence is constantly evolving, with new breakthroughs and challenges emerging every day. Staying informed about the latest news and developments is crucial for understanding the impact of AI on society.

**AI Regulation and Policy: Navigating the Ethical Maze**

As AI becomes more pervasive, governments and organizations are grappling with the need to regulate AI and address ethical concerns. The EU AI Act and the White House AI Bill of Rights are examples of ongoing efforts to establish guidelines and standards for the responsible development and deployment of AI. These regulations aim to ensure that AI is used in a way that is fair, transparent, and accountable.

**AI Funding and Investment: Fueling the Future**

The AI sector is attracting significant investment, with venture capitalists and corporations pouring billions of dollars into AI startups and research. Recent funding rounds and acquisitions in the AI space highlight the growing confidence in the potential of AI to transform industries and create new opportunities.

**AI Ethics and Bias: Addressing the Fairness Challenge**

AI bias is a major concern, as AI systems can perpetuate and amplify existing biases in data, leading to discriminatory outcomes. Researchers and developers are working on strategies for developing fair and ethical AI systems, including techniques for mitigating bias in AI datasets and algorithms.

**AI Job Market Trends: The Rise of the AI Professional**

The demand for AI professionals is rapidly growing, with companies across industries seeking skilled data scientists, machine learning engineers, and AI researchers. Emerging career paths in the field of AI include AI ethics officers, AI trainers, and AI explainability specialists.

## AI Across Industries: Transforming the Way We Work

Artificial Intelligence is no longer a futuristic concept; it's a present-day reality that's transforming industries across the board. From healthcare to finance to marketing, AI is driving innovation and creating new opportunities.

**Healthcare: Revolutionizing Patient Care**

AI is revolutionizing healthcare, enabling AI-powered diagnostics, drug discovery, and personalized medicine. AI-powered medical imaging analysis can help doctors detect diseases earlier and more accurately, while virtual assistants can provide personalized patient care and support.

**Finance: Enhancing Efficiency and Security**

In the finance industry, AI is being used for fraud detection, algorithmic trading, and risk management. AI-powered credit scoring can help lenders make more informed decisions, while automated financial advisors can provide personalized investment advice.

**Marketing: Personalizing the Customer Experience**

AI is transforming marketing by enabling personalized advertising, customer segmentation, and chatbots. AI-powered email marketing can deliver targeted messages to customers based on their individual preferences, while social media analytics can provide insights into customer behavior.

**Manufacturing: Optimizing Production and Quality**

AI is optimizing manufacturing processes through predictive maintenance, quality control, and robotics. AI-powered defect detection can identify flaws in products before they reach customers, while autonomous robots can automate assembly line tasks.

**Transportation: Paving the Way for Autonomous Vehicles**

AI is at the heart of self-driving cars, traffic management systems, and logistics optimization. Autonomous trucks can improve efficiency and safety in transportation, while AI-powered route planning can optimize delivery routes.

## Ethical Considerations and Challenges: Navigating the Risks

While Artificial Intelligence offers tremendous potential, it also raises important ethical considerations and challenges that must be addressed to ensure its responsible development and deployment.

**Bias and Fairness: Ensuring Equitable Outcomes**

AI systems can perpetuate and amplify existing biases in data, leading to discriminatory outcomes. It is crucial to mitigate bias in AI datasets and algorithms to ensure that AI systems are fair and equitable.

**Job Displacement: Preparing for the Future of Work**

AI-driven automation has the potential to displace workers in certain industries. It is important to reskill and upskill workers for the future of work, providing them with the skills they need to thrive in an AI-powered economy.

**Privacy and Security: Protecting Sensitive Data**

AI-powered data collection and analysis can raise privacy concerns. It is essential to protect sensitive data and ensure data security in AI systems.

**Autonomous Weapons: Preventing Unintended Consequences**

The development of autonomous weapons systems raises serious ethical concerns. It is important to have international agreements and regulations in place to prevent the unintended consequences of autonomous weapons.

## The Future of AI: A Glimpse into Tomorrow

The future of AI is full of possibilities, with the potential to transform society in profound ways. However, it is also important to be aware of the challenges that need to be overcome to ensure the responsible development and deployment of AI.

**Predictions and Projections: Imagining the Unimaginable**

Experts predict that AI will continue to advance rapidly, with new breakthroughs in areas like general-purpose AI, brain-computer interfaces, and quantum computing. The long-term impact of AI on society is difficult to predict, but it is likely to be transformative.

**Opportunities and Challenges: Embracing Innovation Responsibly**

The AI field offers tremendous opportunities for innovation and growth. However, it is important to address the challenges of bias, job displacement, privacy, and security to ensure that AI benefits all of humanity.

## Embrace the AI Revolution: Your Call to Action

Artificial Intelligence is no longer a futuristic concept; it's a present-day reality that's shaping our world. Now is the time to explore AI further and understand its potential impact on your life and work.

**Take Action Today:**

*   **Learn more about AI:** Explore online courses, articles, and resources to deepen your understanding of AI concepts and technologies.
*   **Experiment with AI tools:** Try out AI platforms and tools to experience the power of AI firsthand.
*   **Consider a career in AI:** Explore the growing demand for AI professionals and consider pursuing a career in this exciting field.
*   **Engage in ethical discussions:** Participate in conversations about the ethical implications of AI and contribute to the development of responsible AI practices.

**Explore These Resources:**

*   AI Courses: [Links to AI courses]
*   AI Tools: [Links to AI tools and platforms]
*   AI Communities: [Links to AI communities and forums]
*   AI Experts: [Contact information for AI experts or consultants]

Share your thoughts and experiences with AI in the comments section below! Let's learn and grow together in this exciting new era of Artificial Intelligence.
```

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [ ]:
topic = "YOUR TOPIC HERE"
result = crew.kickoff(inputs={"topic": topic})

In [ ]:
Markdown(result)

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).